In [1]:
!pip install pandas
!pip install konlpy
!apt-get install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

!apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

!apt-get install mecab-ko-dic

!echo `mecab-config --dicdir`"/mecab-ko-dic"


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [9]:
import pandas as pd
'''
Cleaning_V1
'…', '...', '·' 제거 후 strip() 적용
'''
file_path = "../data/train_origin.csv"
df = pd.read_csv(file_path)

def clean_text(text):
    # Replace '… ', '...', and '·' with ' ' and then apply strip()
    cleaned_text = text.replace('…', ' ').replace('...', ' ').replace('..', ' ').replace('·', ' ').strip()
    return cleaned_text

df['text'] = df['text'].apply(clean_text)

output_file_path = '../data/train.csv'
df.to_csv(output_file_path, index=False)

In [3]:
import pandas as pd
import re

'''
Cleaning_V3
모든 숫자를 0으로 치환 masking
예) 2016 -> 0000, 64 -> 00
'''
file_path = "../data/train_origin.csv"
df = pd.read_csv(file_path)

def clean_text_v3(text):
    # 모든 숫자를 '0'으로 치환
    cleaned_text = re.sub(r'\d', '0', text)
    return cleaned_text

df['text'] = df['text'].apply(clean_text_v3)

output_file_path = '../data/train_cleaning_v3.csv'
df.to_csv(output_file_path, index=False)

In [29]:
from konlpy.tag import Mecab
import pandas as pd
import re

'''
Cleaning_V4
V2의 개선 버전
명사, 동사만 남기고 모든 조사, 부사 등을 제거함
기존에 붙어 있던 명사는 붙이고, 떨어져 있던 명사는 떨어트림
'''
def extract_nouns(sentence):
    mecab = Mecab('/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ko-dic')
    tokens = mecab.pos(sentence)

    # Filter out unwanted tokens (keeping nouns, numbers, and English words)
    filtered_tokens = [word for word, pos in tokens if pos.startswith('N') or pos.startswith('V') or pos == 'SN' or pos == 'SL']
    result_tokens = []
    
    current_token = ''    
    next_token = ''
    for i in range(len(filtered_tokens) - 1):
        current_token = filtered_tokens[i] if current_token == '' else current_token
        next_token = filtered_tokens[i + 1]
        
        pattern = re.escape(current_token + next_token)
        if re.search(pattern, sentence):
            current_token += next_token
        else:
            result_tokens.append(current_token)
            current_token = ''
            
    if current_token:
        result_tokens.append(current_token)
    else:
        result_tokens.append(next_token)
        
    if len(filtered_tokens) == 1:
        return filtered_tokens
    
    return result_tokens

file_path = "../data/crawling_v7.csv"
output_file_path = "../data/train.csv"
df = pd.read_csv(file_path)
# Create a temporary DataFrame with the same structure as df
temp_df = df.copy()

# Apply extract_nouns to all rows of temp_df['text']
temp_df['nouns'] = temp_df['text'].apply(extract_nouns)

# Join the extracted nouns with spaces in temp_df
temp_df['joined_nouns'] = temp_df['nouns'].apply(lambda x: " ".join(x))

# Overwrite the 'text' column in the original DataFrame (df) with the joined nouns
df['text'] = temp_df['joined_nouns']

df.to_csv(output_file_path, index=False)

In [5]:
import pandas as pd
import re

'''
Cleaning_V6 with Regular Expressions
모든 특수문자 제거
'''
file_path = "../data/train_origin.csv"
df = pd.read_csv(file_path)

def clean_text_v6(text):
    # 특수문자 제거
    cleaned_text = re.sub(r'[^\w\sㄱ-ㅎ가-힣a-zA-Z0-9]', ' ', text)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

df['text'] = df['text'].apply(clean_text_v6)

output_file_path = '../data/train_cleaning_v6.csv'
df.to_csv(output_file_path, index=False)

In [27]:
import pandas as pd
from transformers import AutoTokenizer
'''
Cleaning_V7
모든 [UNK] 토큰 제거
'''
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

file_path = "../data/crawling.csv"
df = pd.read_csv(file_path)

def clean_text_v7(text):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)
    
    # Remove [UNK] tokens
    tokens = [token for token in tokens if token != '[UNK]']
    
    # Reconstruct the cleaned text
    cleaned_text = tokenizer.convert_tokens_to_string(tokens)
    
    return cleaned_text

# Apply the cleaning function to the 'text' column
df['text'] = df['text'].apply(clean_text_v7)

output_file_path = '../data/crawling_v7.csv'
df.to_csv(output_file_path, index=False)